In [1]:
#Könyvtár behívása
import pandas as pd

In [2]:
#Adathalmaz beolvasása
root = r"bank+marketing\bank\bank-full.csv"
ah = pd.read_csv(root, sep=';')

In [ ]:
#Adathalmaz elemzése
print(f"Sorok: {len(ah)}")
print(f"Változók (oszlopok száma): {len(ah.columns)}")
print(ah.head())
print(ah.describe())
print(ah['job'].value_counts())
print(ah['balance'].describe())

Sorok: 45211
Változók (oszlopok száma): 17
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  
                age        balance           day      duration      

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class XMeans:
    
    # X-Means klaszterezési algoritmus implementációja
    # Automatikusan meghatározza az optimális klaszterszámot BIC alapján
    
    
    def __init__(self, kmin=2, kmax=10, random_state=42):
        """
        Paraméterek:
        -----------
        kmin : int
            Minimális klaszterszám
        kmax : int
            Maximális klaszterszám
        random_state : int
            Véletlenszám generátor seed
        """
        self.kmin = kmin
        self.kmax = kmax
        self.random_state = random_state
        self.labels_ = None
        self.cluster_centers_ = None
        self.n_clusters_ = None
        self.bic_scores_ = []
        
    def _calculate_bic(self, X, labels, centers):
        """
        BIC (Bayesian Information Criterion) számítása
        """
        n_samples, n_features = X.shape
        n_clusters = len(centers)
        
        # Számítsuk ki a log-likelihood-ot
        variance = 0
        for i in range(n_clusters):
            cluster_points = X[labels == i]
            if len(cluster_points) > 0:
                variance += np.sum((cluster_points - centers[i]) ** 2)
        
        variance = variance / (n_samples - n_clusters)
        
        if variance == 0:
            variance = 1e-10
            
        log_likelihood = -n_samples * np.log(2 * np.pi) / 2 - \
                        n_samples * n_features * np.log(variance) / 2 - \
                        (n_samples - n_clusters) / 2
        
        # BIC = log(n) * k - 2 * log_likelihood
        # ahol k a paraméterek száma
        n_parameters = n_clusters * n_features + n_clusters  # középpontok + klaszter méretek
        bic = n_parameters * np.log(n_samples) - 2 * log_likelihood
        
        return bic
    
    def fit(self, X):
        
        # X-Means illesztése az adatokra
        
        best_bic = np.inf
        best_k = self.kmin
        
        print("X-Means klaszterezés futtatása...")
        print("=" * 60)
        
        # Próbáljuk ki az összes k értéket kmin-től kmax-ig
        for k in range(self.kmin, self.kmax + 1):
            kmeans = KMeans(n_clusters=k, random_state=self.random_state, n_init=10)
            labels = kmeans.fit_predict(X)
            centers = kmeans.cluster_centers_
            
            bic = self._calculate_bic(X, labels, centers)
            self.bic_scores_.append({'k': k, 'bic': bic})
            
            print(f"k = {k}: BIC = {bic:.2f}")
            
            # Keressük a legkisebb BIC értéket (jobb a kisebb)
            if bic < best_bic:
                best_bic = bic
                best_k = k
                self.labels_ = labels
                self.cluster_centers_ = centers
        
        self.n_clusters_ = best_k
        print("=" * 60)
        print(f"Optimális klaszterszám: {self.n_clusters_}")
        print(f"Legjobb BIC érték: {best_bic:.2f}")
        
        return self
    
    def predict(self, X):
        """
        Új adatpontok klaszterekhez rendelése
        """
        distances = np.sqrt(((X[:, np.newaxis] - self.cluster_centers_) ** 2).sum(axis=2))
        return np.argmin(distances, axis=1)


In [5]:
# Adatok generálása (ugyanaz, mint a K-Means példában)
np.random.seed(42)

cluster_params = [
    {'mean_age': 31.18, 'mean_balance': 859.11, 'n_samples': 11000},
    {'mean_age': 56.22, 'mean_balance': 1325.48, 'n_samples': 8000},
    {'mean_age': 43.06, 'mean_balance': 14019.78, 'n_samples': 2000},
    {'mean_age': 42.36, 'mean_balance': 905.16, 'n_samples': 24211}
]

data_list = []
for i, params in enumerate(cluster_params):
    ages = np.random.normal(params['mean_age'], 8, params['n_samples'])
    balances = np.random.normal(params['mean_balance'], 2000, params['n_samples'])
    
    cluster_df = pd.DataFrame({
        'age': ages,
        'balance': balances
    })
    data_list.append(cluster_df)

ah = pd.concat(data_list, ignore_index=True)

# Adatok standardizálása
X = ah[['age', 'balance']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# X-Means klaszterezés
xmeans = XMeans(kmin=2, kmax=8, random_state=42)
xmeans.fit(X_scaled)

# Címkék hozzáadása az eredeti adatokhoz
ah['cluster'] = xmeans.labels_

# Középpontok visszaalakítása az eredeti skálára
centroids_original = scaler.inverse_transform(xmeans.cluster_centers_)

print("\n" + "=" * 60)
print("KLASZTER STATISZTIKÁK")
print("=" * 60)
cluster_stats = ah.groupby('cluster').agg({
    'age': ['mean', 'std', 'count'],
    'balance': ['mean', 'std']
}).round(2)
print(cluster_stats)

print("\n" + "=" * 60)
print("KLASZTER KÖZÉPPONTOK (eredeti skálán)")
print("=" * 60)
for i, (age, balance) in enumerate(centroids_original):
    count = len(ah[ah['cluster'] == i])
    percentage = (count / len(ah)) * 100
    print(f"{i+1}. klaszter: Életkor = {age:.2f} év, Egyenleg = {balance:.2f} €, "
          f"Méret = {count} ({percentage:.2f}%)")

X-Means klaszterezés futtatása...
k = 2: BIC = 154268.47
k = 3: BIC = 99444.46
k = 4: BIC = 75145.20
k = 5: BIC = 52721.49
k = 6: BIC = 38487.25
k = 7: BIC = 25594.07
k = 8: BIC = 15749.74
Optimális klaszterszám: 8
Legjobb BIC érték: 15749.74

KLASZTER STATISZTIKÁK
           age               balance         
          mean   std count      mean      std
cluster                                      
0        34.96  3.94  6338  -1239.23  1232.71
1        48.92  4.03  6100   3147.39  1227.12
2        23.67  4.57  4965    802.31  1520.52
3        43.39  7.92  1996  14084.43  1986.77
4        61.97  4.83  4583   1488.20  1603.36
5        42.43  2.83  8146    746.10   870.36
6        34.40  3.74  6984   2778.93  1246.84
7        50.96  3.92  6099   -894.53  1274.96

KLASZTER KÖZÉPPONTOK (eredeti skálán)
1. klaszter: Életkor = 34.99 év, Egyenleg = -1241.62 €, Méret = 6338 (14.02%)
2. klaszter: Életkor = 48.88 év, Egyenleg = 3153.32 €, Méret = 6100 (13.49%)
3. klaszter: Életkor = 23.67 év, E

In [ ]:
#import matplotlib.pyplot as plt
#from scipy import stats


# Mentés CSV-be
ah.to_csv('/mnt/user-data/outputs/xmeans_results.csv', index=False, encoding='utf-8-sig')
print(f"\n✓ Eredmények mentve: xmeans_results.csv")

# BIC értékek DataFrame-be
bic_df = pd.DataFrame(xmeans.bic_scores_)
bic_df.to_csv('/mnt/user-data/outputs/xmeans_bic_scores.csv', index=False)
print(f"✓ BIC értékek mentve: xmeans_bic_scores.csv")